In [70]:
import os
import json
import pandas as pd

# Define the path to the main folder
parent_folder_path = '/home/scur2012/Thesis/master-thesis/experiments/tuning/ray_results/'

all_df = {}
for run in ['transformer_t', 'transformer_f', 'transformer_combined', 'transformer_mvts' ]:

    # Initialize an empty list to hold the data
    data_list = []

    main_folder_path = parent_folder_path + run
    
    # Loop through each folder in the main folder
    for folder_name in os.listdir(main_folder_path):
        folder_path = os.path.join(main_folder_path, folder_name)
        
        if os.path.isdir(folder_path):
            # Find the JSON and CSV files in the folder
            json_file_path = os.path.join(folder_path, 'params.json')
            csv_file_path = os.path.join(folder_path, 'progress.csv')
            
            if os.path.isfile(json_file_path) and os.path.isfile(csv_file_path):
                # Load the JSON file
                with open(json_file_path, 'r') as json_file:
                    json_data = json.load(json_file)
                
                # Load the CSV file and get the last row
                csv_data = pd.read_csv(csv_file_path)
                last_row = csv_data.iloc[-1]
                
                # Combine the JSON data and the last row of the CSV into a single dictionary
                combined_data = {**json_data, **last_row.to_dict()}
                
                # Append the combined data to the list
                data_list.append(combined_data)

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data_list)
    df = df.drop(columns=[
        'timestamp', 'checkpoint_dir_name', 'should_checkpoint', 'done', 'trial_id', 
        'date', 'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore', 
        'iterations_since_restore', 'max_num_epochs'
    ])
    
    df = df.sort_values(by=['training_iteration', 'loss', 'time_this_iter_s'], ascending=[False, True, True])
    duplicate_colums = [col for col in df.columns if col not in ['training_iteration', 'loss', 'time_this_iter_s']]
    df = df.drop_duplicates(subset = duplicate_colums)
    all_df[run] = df[duplicate_colums + ['training_iteration', 'time_this_iter_s', 'loss']]
    
for k,v in all_df.items():
    print (k)
    print (v.head().to_string())

transformer_t
    batch_size  depth  dim  dropout  emb_dropout  heads  learning_rate  mlp_dim  num_classes  patch_size  training_iteration  time_this_iter_s      loss
85          64      7  512      0.0          0.0      7        0.00016     1024           16          16                  20        125.578363 -2.808831
75          64      7  512      0.0          0.0      6        0.00016     1024           24          16                  20        114.581814 -2.799606
9           64      7  512      0.0          0.0      6        0.00016     1024           32          16                  20        116.160428 -2.792183
5           64      7  512      0.0          0.0      6        0.00016     2048           24          16                  17        146.934338 -2.643052
83          64      7  512      0.0          0.0      6        0.00012     2048           24          16                  10        148.480175 -2.410707
transformer_f
    batch_size  depth   dim  dropout  emb_dropout  hea

In [96]:
run = 'transformer_mvts'
df = all_df[run].copy()

if run == 'transformer_combined':
    df['mlp_dim_f'] = df['mlp_dim_f'].fillna(1024)
    df['mlp_dim_f'] = df['mlp_dim_f'].astype(int)
    
# Apply formatting
try:
    df['dropout'] = df['dropout'].apply(lambda x: f"{x:.2f}")
    df['emb_dropout'] = df['emb_dropout'].apply(lambda x: f"{x:.2f}")
except KeyError:
    pass

df['learning_rate'] = df['learning_rate'].apply(lambda x: f"{x:.2e}")
df['loss'] = df['loss'].apply(lambda x: f"{x:.2f}")
df['time_this_iter_s'] = df['time_this_iter_s'].astype(int)



# rename columns
col_newnames = dict(
    batch_size = "Batch size",
    depth  = "Depth",
    dim = "Dimensions",
    dropout = "Dropout",
    emb_dropout = "EMB dropout",
    heads = "Num heads",
    learning_rate = "LR",
    mlp_dim = "MLP dim",
    mlp_dim_f = "MLP dim f",
    num_classes = "Num classes",
    num_classes_t = "Num classes t",
    num_classes_f = "Num classes f",
    patch_size = "Patch size",
    training_iteration  = "Epochs",
    time_this_iter_s  = "Time per epoch (s)",
    loss = "Loss",
    d_model = "Dimensions",
    dim_feedforward = "MLP dim",
    n_heads = "Num heads",
    num_layers = "Depth",
    pos_encoding = "Positional encoding"
)

df = df.rename(columns=col_newnames)

top_10 = df.head(10)
bottom_10 = df.tail(10)

latex_body = pd.concat([top_10, bottom_10]).to_latex(index=False, caption=run)

print (latex_body)


\begin{table}
\caption{transformer_mvts}
\begin{tabular}{rrrllrrlrrl}
\toprule
Batch size & Dimensions & MLP dim & Dropout & LR & Num heads & Depth & Positional encoding & Epochs & Time per epoch (s) & Loss \\
\midrule
32 & 128 & 1024 & 0.00 & 1.00e-04 & 4 & 6 & fixed & 10 & 67 & -2.43 \\
32 & 128 & 1024 & 0.00 & 1.00e-04 & 4 & 5 & fixed & 10 & 59 & -2.40 \\
32 & 128 & 512 & 0.00 & 1.00e-04 & 8 & 3 & fixed & 10 & 50 & -2.20 \\
32 & 128 & 512 & 0.05 & 1.00e-04 & 16 & 3 & learnable & 10 & 77 & -1.93 \\
64 & 128 & 256 & 0.10 & 1.00e-03 & 8 & 3 & learnable & 10 & 44 & -0.31 \\
64 & 128 & 1024 & 0.00 & 1.29e-04 & 4 & 6 & fixed & 7 & 58 & -2.20 \\
32 & 128 & 512 & 0.00 & 1.00e-04 & 8 & 6 & fixed & 7 & 77 & -2.17 \\
32 & 128 & 1024 & 0.00 & 1.00e-04 & 8 & 5 & fixed & 7 & 75 & -2.17 \\
32 & 128 & 1024 & 0.00 & 1.00e-04 & 4 & 7 & fixed & 7 & 75 & -2.12 \\
64 & 256 & 512 & 0.00 & 1.00e-04 & 4 & 6 & fixed & 7 & 73 & -2.08 \\
32 & 128 & 1024 & 0.00 & 9.80e-04 & 4 & 6 & fixed & 1 & 67 & 0.01 \\
32 

In [72]:
print (df_filtered.to_string())

    batch_size  learning_rate  num_classes_t  num_classes_f  training_iteration  time_this_iter_s      loss
30          64       0.000160             16             16                  14        187.808537 -2.577608
34          64       0.000160             28             16                   6        162.427100 -2.101107
60          64       0.000160             28             12                   6        162.773312 -2.090942
65          64       0.000154             20             17                   6        189.149701 -2.059628
51          64       0.000160             32             32                   5        178.628841 -2.013043
24          64       0.000105             19             17                   5        189.590113 -2.012775
61          64       0.000135             16             15                   5        189.140957 -1.984520
67          64       0.000104             13             16                   5        188.695028 -1.982316
57          64       0.00016